In [8]:
# coding: utf-8
from urllib import request, parse
import ssl
import json
import time

# 配置变量
config = {
    # 用户ID
    'userId':'',  # TODO, 你的用户id，PC上在币乎页面，按F12，network页面找到对应的请求，可以找到
    # 登录token
    'accessToken':'' # TODO, 你的token，登录后币乎服务器返回，获取方法与userId相同。
}

def datetime_str():  
    format = '%Y-%m-%d %H:%M:%S'  
    #format = '%Y-%m-%d %H:%M:%S'  
    #value 为时间戳值,如:1460073600.0  
    value = time.localtime(time.time())
    dt = time.strftime(format,value)  
    return dt

def print_info(info):
    log = "[%s]--%s" % (datetime_str(), info)
    print(log)

# 取消关注某人    
def unFollow(subjectUserId, userName):
    print_info("取消关注【%s】..." % (userName))
    url = r'https://be02.bihu.com/bihube-pc/api/content/unFollow'
    headers = {
        'User-Agent': r'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
        'Content-Type': r'application/x-www-form-urlencoded;charset=utf-8',
        'Referer': r'https://bihu.com/people/' + config['userId'] + r'/index=2',
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9'
    }
    
    data = {
        'userId': config['userId'],
        'accessToken': config['accessToken'],
        'subjectUserId': subjectUserId
    }
    context = ssl._create_unverified_context()
    data = parse.urlencode(data).encode('utf-8')
    req = request.Request(url, headers=headers, data=data)

    rsp = ""
    try:
        rsp = request.urlopen(req, context = context).read()
        rsp = rsp.decode('utf-8')
    except urllib2.URLError as e:
        print_info(str(e.code) + '\r\n' + e.reason);

    print_info("取消关注【%s】返回：%s" % (userName, rsp))

    return rsp

# 获取关注列表，pageNum为页序    
def getUserFollowList(pageNum):
    print_info("查询第【" + str(pageNum) + "】页关注列表" )
    url = r'https://be02.bihu.com/bihube-pc/api/content/show/getUserFollowList'
    headers = {
        'User-Agent': r'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
        'Content-Type': r'application/x-www-form-urlencoded;charset=utf-8',
        'Referer': r'https://bihu.com/people/' + config['userId'] + r'/index=2',
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9'
    }
    
    data = {
        'queryUserId': config['userId'],
        'userId': config['userId'],
        'accessToken': config['accessToken'],
        'pageNum': pageNum
    }
    
    context = ssl._create_unverified_context()
    data = parse.urlencode(data).encode('utf-8')
    req = request.Request(url, headers=headers, data=data)

    rsp = ""
    try:
        rsp = request.urlopen(req, context = context).read()
        rsp = rsp.decode('utf-8')
    except urllib2.URLError as e:
        print_info(str(e.code) + '\r\n' + e.reason);

    return rsp

def parse_follow_list(body):
    data_str = json.loads(body);
    res = data_str['res'];

    # 请求是否成功
    is_succ = False;
    pages = 0;
    log_content_format = '【%s】的粉丝数：%d，头像是:%s'
    if res == 1 and data_str["data"]["size"] > 0:
        pages = data_str["data"]["pages"]
        # 遍历关注对象
        for item in data_str["data"]["list"]:
            if item["userName"].startswith("币友_") \
            or item["userIcon"] == "img/bihu_user_default_icon.png"\
            or item["fans"] < 500:
                log_content = log_content_format % (item['userName'], item["fans"], item["userIcon"])
                print_info(log_content)
                # 昵称以币友_开始，userIcon为默认，且粉丝数小于500的，取消关注
                unFollow(item["userId"],item['userName'])
    
    if res == 1:
        is_succ  = True;

    return is_succ, pages

def Run():
    # 第一次获取，为了拿到页数，好做循环
    pageIndex = 1
    body = getUserFollowList(pageIndex)
    is_suc, pages = parse_follow_list(body)

    while is_suc and pageIndex < pages:
        pageIndex += 1
        time.sleep(10)
        body = getUserFollowList(pageIndex)
        is_suc, pages = parse_follow_list(body)

    if (not is_suc):
        print_info(body)
        print_info("请稍后重试")
        
Run()
print_info("程序已退出")

[2018-04-14 00:01:01]--查询第【1】页关注列表
[2018-04-14 00:01:01]--【孙小微】的粉丝数：38，头像是:img/b1e4cfcdab334bf7c17d950af338e5aa.jpeg
[2018-04-14 00:01:01]--取消关注【孙小微】...
[2018-04-14 00:01:02]--取消关注【孙小微】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:02]--【在家说英语】的粉丝数：287，头像是:img/b8b128f599622dc8cf1b267b3b1b5a2e.jpeg
[2018-04-14 00:01:02]--取消关注【在家说英语】...
[2018-04-14 00:01:02]--取消关注【在家说英语】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:02]--【币圈喵叔】的粉丝数：46，头像是:img/a6102954ef7ed8d5e2b7058f0891d084.jpg
[2018-04-14 00:01:02]--取消关注【币圈喵叔】...
[2018-04-14 00:01:02]--取消关注【币圈喵叔】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:02]--【MasterKing】的粉丝数：164，头像是:img/ca80b2594a865971888cc2de1a6bf63e.jpeg
[2018-04-14 00:01:02]--取消关注【MasterKing】...
[2018-04-14 00:01:02]--取消关注【MasterKing】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:02]--【东本聪】的粉丝数：84，头像是:img/64336fecb2afecb66817394f938f4093.jpeg
[2018-04-14 00:01:02]--取消关注【东本聪】...
[2018-04-14 00:01:02]--取消关注【东本聪】返回：{"dat

[2018-04-14 00:01:31]--取消关注【笑对人生213942】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:31]--【世界比特】的粉丝数：268，头像是:img/0b23239f02f58e2aef535979e6098acf.jpeg
[2018-04-14 00:01:31]--取消关注【世界比特】...
[2018-04-14 00:01:32]--取消关注【世界比特】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:32]--【轩三少】的粉丝数：110，头像是:img/b108f9b4ecf4e7660e234218b8f52b5d.jpeg
[2018-04-14 00:01:32]--取消关注【轩三少】...
[2018-04-14 00:01:32]--取消关注【轩三少】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:32]--【Robinhood】的粉丝数：116，头像是:img/54a184c5635a9fd0822d017b81a4742b.jpeg
[2018-04-14 00:01:32]--取消关注【Robinhood】...
[2018-04-14 00:01:32]--取消关注【Robinhood】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:32]--【Dr杨杨】的粉丝数：251，头像是:img/6d58097fd44d7743566403641c7164e0.jpeg
[2018-04-14 00:01:32]--取消关注【Dr杨杨】...
[2018-04-14 00:01:32]--取消关注【Dr杨杨】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:01:42]--查询第【4】页关注列表
[2018-04-14 00:01:42]--【牛仔】的粉丝数：419，头像是:img/44d1e08ad40d06c94813717750cd976c.j

[2018-04-14 00:02:21]--取消关注【张小样】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:21]--【美欣妹妹】的粉丝数：92，头像是:img/b0e7d9ab11012f142d3faf442de064ac.jpeg
[2018-04-14 00:02:21]--取消关注【美欣妹妹】...
[2018-04-14 00:02:22]--取消关注【美欣妹妹】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:22]--【原生态韭菜】的粉丝数：66，头像是:img/1f00fe0d0abba4b1ed327120c083c7fa.jpeg
[2018-04-14 00:02:22]--取消关注【原生态韭菜】...
[2018-04-14 00:02:22]--取消关注【原生态韭菜】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:22]--【宁汪汪】的粉丝数：105，头像是:img/bc66665291a32fa37eb84744ec5a25cc.jpg
[2018-04-14 00:02:22]--取消关注【宁汪汪】...
[2018-04-14 00:02:22]--取消关注【宁汪汪】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:22]--【何怕冷】的粉丝数：80，头像是:img/cfc8a8a2079b3678a437042cd11be8bb.jpeg
[2018-04-14 00:02:22]--取消关注【何怕冷】...
[2018-04-14 00:02:22]--取消关注【何怕冷】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:22]--【新鲜的韭菜】的粉丝数：125，头像是:img/876f95304a6247498653537041fa5b32.jpeg
[2018-04-14 00:02:22]--取消关注【新鲜的韭菜】...
[2018-04-14 00:0

[2018-04-14 00:02:52]--取消关注【七化】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:52]--【aperia】的粉丝数：227，头像是:img/fc6494b8931b3a396cbcc20692bd7a62.jpeg
[2018-04-14 00:02:52]--取消关注【aperia】...
[2018-04-14 00:02:52]--取消关注【aperia】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:52]--【仙人掌】的粉丝数：127，头像是:img/ee664734f62af7322a1ba39ff3a1295a.jpg
[2018-04-14 00:02:52]--取消关注【仙人掌】...
[2018-04-14 00:02:52]--取消关注【仙人掌】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:52]--【小miner】的粉丝数：374，头像是:img/05fd9b4b881ce7eb782182392f124b9e.jpeg
[2018-04-14 00:02:52]--取消关注【小miner】...
[2018-04-14 00:02:52]--取消关注【小miner】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:52]--【车南】的粉丝数：390，头像是:img/1f1f55d3972fde0b982c59a48d992e3f.jpeg
[2018-04-14 00:02:52]--取消关注【车南】...
[2018-04-14 00:02:52]--取消关注【车南】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:02:52]--【尖毛草】的粉丝数：30，头像是:img/7d52d2eeb657c5a9652ef3faa55ffa21.jpeg
[2018-04-14 00:02:52]--取消关注【尖毛草】...
[2018-04-14 0

[2018-04-14 00:03:22]--取消关注【非韭】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:03:22]--【二条说币】的粉丝数：127，头像是:img/73591b32d908ef0318418e312c5724ea.jpeg
[2018-04-14 00:03:22]--取消关注【二条说币】...
[2018-04-14 00:03:22]--取消关注【二条说币】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:03:22]--【Gilgamesh】的粉丝数：92，头像是:img/18ff3a627722bbbfcd94617ad388cb2a.jpeg
[2018-04-14 00:03:22]--取消关注【Gilgamesh】...
[2018-04-14 00:03:22]--取消关注【Gilgamesh】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:03:22]--【龙少】的粉丝数：94，头像是:img/0840d8645c1be535b4ac69b1812246ce.jpeg
[2018-04-14 00:03:22]--取消关注【龙少】...
[2018-04-14 00:03:22]--取消关注【龙少】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:03:32]--查询第【12】页关注列表
[2018-04-14 00:03:32]--【币友_14317】的粉丝数：68，头像是:img/bihu_user_default_icon.png
[2018-04-14 00:03:32]--取消关注【币友_14317】...
[2018-04-14 00:03:33]--取消关注【币友_14317】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:03:33]--【币友_20758】的粉丝数：210，头像是:img/bihu_user_default_icon.png
[2018-04-14 0

...

[2018-04-14 00:09:42]--取消关注【小样VR】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:09:42]--【币界洪流】的粉丝数：315，头像是:img/5aedac37697b5e7820356b35e323ffa3.jpg
[2018-04-14 00:09:42]--取消关注【币界洪流】...
[2018-04-14 00:09:42]--取消关注【币界洪流】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:09:42]--【币定赢】的粉丝数：144，头像是:img/c5f1ff34c8f5b41d0aabbefcd4ed8ae1.jpeg
[2018-04-14 00:09:42]--取消关注【币定赢】...
[2018-04-14 00:09:43]--取消关注【币定赢】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:09:43]--【口木山人】的粉丝数：128，头像是:img/c1791414c13384c9a581a8ae04c2bff8.JPG
[2018-04-14 00:09:43]--取消关注【口木山人】...
[2018-04-14 00:09:43]--取消关注【口木山人】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:09:43]--【蘋果你個挨炮】的粉丝数：158，头像是:img/954e848c52627401a56f8062dfa41570.jpeg
[2018-04-14 00:09:43]--取消关注【蘋果你個挨炮】...
[2018-04-14 00:09:43]--取消关注【蘋果你個挨炮】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:09:43]--【光海】的粉丝数：374，头像是:img/240653118ad058891fc8213a18e74ebe.jpeg
[2018-04-14 00:09:43]--取消关注【光海】...
[2018-04-14 0

[2018-04-14 00:10:23]--取消关注【gaojiaqi】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:10:23]--【比特无极】的粉丝数：309，头像是:img/0cdd2d2ec9c44c848c287a94d369eab6.jpeg
[2018-04-14 00:10:23]--取消关注【比特无极】...
[2018-04-14 00:10:23]--取消关注【比特无极】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:10:23]--【Wynn稳赢】的粉丝数：384，头像是:img/d88b5358529656d1ede5915e56912e0f.jpeg
[2018-04-14 00:10:23]--取消关注【Wynn稳赢】...
[2018-04-14 00:10:23]--取消关注【Wynn稳赢】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:10:23]--【币世界二吹】的粉丝数：215，头像是:img/2f04a74da95e40afc74b8023d24b96a3.jpeg
[2018-04-14 00:10:23]--取消关注【币世界二吹】...
[2018-04-14 00:10:23]--取消关注【币世界二吹】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:10:23]--【soupercj】的粉丝数：245，头像是:img/70b4e8d2bb6b5fb25c3fe5d48ecc6852.jpeg
[2018-04-14 00:10:23]--取消关注【soupercj】...
[2018-04-14 00:10:24]--取消关注【soupercj】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:10:24]--【张小兰】的粉丝数：212，头像是:img/1e3202849a9019d8e5ce3dc7c5e2e0c9.jpeg
[2018-04-14 00:10:24]--

[2018-04-14 00:11:04]--取消关注【飞鹰翱翔】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:11:04]--【王大铭】的粉丝数：451，头像是:img/b26f65a86cd1cc1ca2e20b1ec2393eeb.jpeg
[2018-04-14 00:11:04]--取消关注【王大铭】...
[2018-04-14 00:11:04]--取消关注【王大铭】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:11:04]--【特立独行的猫】的粉丝数：472，头像是:img/340a387f66bd2ffb2153b4b44b93dea1.jpeg
[2018-04-14 00:11:04]--取消关注【特立独行的猫】...
[2018-04-14 00:11:04]--取消关注【特立独行的猫】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:11:04]--【CJN】的粉丝数：215，头像是:img/8ba240f2bf1d32f32bd30c900b88d09b.jpeg
[2018-04-14 00:11:04]--取消关注【CJN】...
[2018-04-14 00:11:04]--取消关注【CJN】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:11:04]--【DMAN】的粉丝数：239，头像是:img/7a087bb5302813afe9618cf8f2b223c1.jpeg
[2018-04-14 00:11:04]--取消关注【DMAN】...
[2018-04-14 00:11:04]--取消关注【DMAN】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:11:04]--【刘岩】的粉丝数：450，头像是:img/da89aeec8e38e79ce987d76b22cc5df5.jpeg
[2018-04-14 00:11:04]--取消关注【刘岩】...
[2018-04-14 00

[2018-04-14 00:12:04]--取消关注【大爱无疆007】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:04]--【韭菜哥哥】的粉丝数：298，头像是:img/ee3fdf42633cbb0749e3cecf1021b2fa.jpeg
[2018-04-14 00:12:04]--取消关注【韭菜哥哥】...
[2018-04-14 00:12:04]--取消关注【韭菜哥哥】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:04]--【SUAOVER】的粉丝数：200，头像是:img/92b4b5ca411c57e491de625f3409fa40.jpeg
[2018-04-14 00:12:04]--取消关注【SUAOVER】...
[2018-04-14 00:12:05]--取消关注【SUAOVER】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:05]--【区块链世界的冬青】的粉丝数：293，头像是:img/b9c38c856dddcec6e87d4e8db6445fb5.jpeg
[2018-04-14 00:12:05]--取消关注【区块链世界的冬青】...
[2018-04-14 00:12:05]--取消关注【区块链世界的冬青】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:05]--【币路相遇】的粉丝数：228，头像是:img/32655e32fdd1e9167f7302f7c6dc178a.jpeg
[2018-04-14 00:12:05]--取消关注【币路相遇】...
[2018-04-14 00:12:05]--取消关注【币路相遇】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:05]--【system】的粉丝数：404，头像是:img/8b36901233f4041669c54bdf65e7ddeb.jpeg
[2018-04-14 00:12:05]

[2018-04-14 00:12:45]--取消关注【林谷冲】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:45]--【尔不语】的粉丝数：349，头像是:img/964b14b0062dc820a649ea4c8674338f.jpeg
[2018-04-14 00:12:45]--取消关注【尔不语】...
[2018-04-14 00:12:45]--取消关注【尔不语】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:45]--【数字淘淘】的粉丝数：304，头像是:img/5c0676415266081f67c4f2aab809c4c7.jpeg
[2018-04-14 00:12:45]--取消关注【数字淘淘】...
[2018-04-14 00:12:45]--取消关注【数字淘淘】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:45]--【币圈小世界】的粉丝数：418，头像是:img/1a8ef8fa7592e01347b9517a7e9812b4.jpeg
[2018-04-14 00:12:45]--取消关注【币圈小世界】...
[2018-04-14 00:12:45]--取消关注【币圈小世界】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:45]--【李伟龙】的粉丝数：361，头像是:img/a9080bb2e323985de9ccfd3ab7139a51.jpeg
[2018-04-14 00:12:45]--取消关注【李伟龙】...
[2018-04-14 00:12:45]--取消关注【李伟龙】返回：{"data":null,"res":1,"resMsg":"success"}
[2018-04-14 00:12:46]--【accumulate】的粉丝数：332，头像是:img/b0848d8bbbbea55c6e700433322b7ec3.jpeg
[2018-04-14 00:12:46]--取消关注【accumulate】...
[2